<a href="https://colab.research.google.com/github/enya-yx/LangChain-Courses/blob/main/LCEL_basic_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install "langchain-google-genai" "langchain" "langchain-core" "langgraph-prebuilt" "google-generativeai" "langchain_community" "docarray" "langchain_experimental"

INFO: pip is looking at multiple versions of google-generativeai to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of google-generativeai to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is looking at multiple versions of langchain to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of langchain to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of langgraph-prebuilt to determine which version is compatible with other requirements. This could t

In [2]:
import google.generativeai as genai
import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('google_api_key')
# Configure the generative AI library with your API key
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])


In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Define llm
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0,
    verbose=True
)


In [8]:
# Define a simple LCEL
prompt = ChatPromptTemplate.from_template(
    "Tell me a short joke about {topic}"
    )
output_parser = StrOutputParser()
chain = prompt | llm | output_parser

In [15]:
#chain.invoke({"topic": "cats"})
chain.batch([{"topic": "cats"}, {"topic": "dear"}])


['Why did the cat sit on the computer?\nBecause it wanted to keep an eye on the mouse!',
 'What do you call a deer with no eyes?\nNo eye-deer!']

In [16]:
for t in chain.stream({"topic": "dogs"}):
  print(t)

Why don't dogs make good dancers?

Because they have two left feet!


In [17]:
response = await chain.ainvoke({"topic": "dinosaur"})
print(response)

Why did the T-Rex cross the road?
...Because it couldn't reach the other side!


In [6]:
from langchain.vectorstores import DocArrayInMemorySearch
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_core.documents import Document

docs = ["Werewolves Kill is a famous boarding game.", "Werewolves Kill normally involves 12 players, with 8 people and 4 wolves."]
# Convert strings to Document objects
docs = [Document(page_content=d) for d in docs]
embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")
db = DocArrayInMemorySearch.from_documents(docs, embeddings)
retriever = db.as_retriever()

In [ ]:
from langchain.schema.runnable import RunnableMap

template = """ Answer the question only based on the following context: {context}\

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)
runnable = RunnableMap({
    "context": lambda x: retriever.invoke(x["question"]),
    "question": lambda x: x["question"]
})
chain = runnable | prompt | llm | StrOutputParser()

In [ ]:
chain.invoke({"question": "How many players can be involved in the Werewolves Kill game?"})


'12 players'

In [ ]:
import json
simple_chain = llm | StrOutputParser()
json_chain = simple_chain | (lambda x: json.loads(x.strip('```json\n').strip('\n```')))

In [ ]:
message = "Provide 3 famous novels in a json blob (not an array), where each novel is a json blob of title, author and short description."

In [ ]:
json_chain.invoke(message)

{'novel1': {'title': 'Pride and Prejudice',
  'author': 'Jane Austen',
  'short_description': 'A classic novel of manners, love, and social class in early 19th-century England, focusing on Elizabeth Bennet and Mr. Darcy.'},
 'novel2': {'title': '1984',
  'author': 'George Orwell',
  'short_description': 'A dystopian social science fiction novel and cautionary tale about the dangers of totalitarianism, surveillance, and thought control.'},
 'novel3': {'title': 'To Kill a Mockingbird',
  'author': 'Harper Lee',
  'short_description': 'A powerful story set in the American South during the 1930s, exploring themes of racial injustice, prejudice, and the loss of innocence through the eyes of young Scout Finch.'}}